Install pdfplumber library to extract text from PDF

In [ ]:
pip install pdfplumber

Import pdfplumber, regex

In [ ]:
import pdfplumber
import re

Keys to be extracted

In [ ]:
keys=['Temperature','Height','Weight','Blood Pressure']

Load File 

In [ ]:
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://github.com/dharsha454/Test-documents/blob/8ad79817bbef0f5dbf1b460ece905657ff03f3bf/Test.pdf

Text Extraction

In [ ]:
with pdfplumber.open(r'Test.pdf') as pdf:
    curr = pdf.pages[0]
    text = (curr.extract_text())

In [ ]:
print(text)

In [ ]:
result={}
j=''

Extracting key value pairs using regex

In [ ]:

for i in keys:
  if i=='Blood Pressure':
    j = re.search(r'{}\w*:\d*\/\d*'.format(i),text).group()

  elif re.search(r'{}\w*:\d*'.format(i),text):
    j = re.search(r'{}\w*:\d*'.format(i),text).group()
   
  elif re.search(r'{}\w*:\d*'.format(i),text):
    j = re.search(r'{}\w*:\d*'.format(i),text).group()

  j= j.replace('\n','')
  j = j.replace(':','') 
  result[i] = j.replace(i,'')

In [ ]:
result

{'Blood Pressure': '130/80',
 'Height': '175',
 'Temperature': '98',
 'Weight': '170'}

Exporting to json

In [ ]:
import json
with open('output.json', 'w') as f:
    json.dump(result,f)

In [ ]:
test_json = json.load(open('output.json'))
test_json

{'Blood Pressure': '130/80',
 'Height': '175',
 'Temperature': '98',
 'Weight': '170'}